### User-Defined Reduction

The `declare` `reduction` directive can be used to specify  user-defined reductions (UDR) for user data types.

In the following example, `declare` `reduction` directives are used to define  _min_  and  _max_  operations for the  _point_  data structure for computing the rectangle that encloses a set of 2-D points.

Each `declare` `reduction` directive defines new reduction identifiers,  _min_  and  _max_ , to be used in a `reduction` clause. The next item in the declaration list is the data type ( _struct_   _point_ ) used in the reduction, followed by the combiner, here the functions  _minproc_  and  _maxproc_  perform the min and max operations, respectively, on the user data (of type  _struct_   _point_ ). In the function argument list are two special OpenMP variable identifiers, `omp_in` and `omp_out`, that denote the two values to be combined in the "real" function; the `omp_out` identifier indicates which one is to hold the result.

The initializer of the `declare` `reduction` directive specifies the initial value for the private variable of each implicit task. The `omp_priv` identifier is used to denote the private variable.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.1.c
* type: C
* version: omp_4.0
*/
#include <stdio.h>
#include <limits.h>

struct point {
  int x;
  int y;
};

void minproc ( struct point *out, struct point *in )
{
  if ( in->x < out->x ) out->x = in->x;
  if ( in->y < out->y ) out->y = in->y;
}

void maxproc ( struct point *out, struct point *in )
{
  if ( in->x > out->x ) out->x = in->x;
  if ( in->y > out->y ) out->y = in->y;
}

#pragma omp declare reduction(min : struct point : \
        minproc(&omp_out, &omp_in)) \
 initializer( omp_priv = { INT_MAX, INT_MAX } )

#pragma omp declare reduction(max : struct point : \
        maxproc(&omp_out, &omp_in)) \
 initializer( omp_priv = { 0, 0 } )

void find_enclosing_rectangle ( int n, struct point points[] )
{
  struct point minp = { INT_MAX, INT_MAX }, maxp = {0,0};
  int i;

#pragma omp parallel for reduction(min:minp) reduction(max:maxp)
  for ( i = 0; i < n; i++ ) {
     minproc(&minp, &points[i]);
     maxproc(&maxp, &points[i]);
  }
  printf("min = (%d, %d)\n", minp.x, minp.y);
  printf("max = (%d, %d)\n", maxp.x, maxp.y);
}



The following example shows the corresponding code in Fortran.  The `declare` `reduction` directives are specified as part of  the declaration in subroutine  _find_enclosing_rectangle_  and  the procedures that perform the min and max operations are specified as subprograms.

In [ ]:

! name: udr.1.f90
! type: F-free
! version: omp_4.0
module data_type

  type :: point
    integer :: x
    integer :: y
  end type

end module data_type

subroutine find_enclosing_rectangle ( n, points )
  use data_type
  implicit none
  integer :: n
  type(point) :: points(*)

  !$omp declare reduction(min : point : minproc(omp_out, omp_in)) &
  !$omp&  initializer( omp_priv = point( HUGE(0), HUGE(0) ) )

  !$omp declare reduction(max : point : maxproc(omp_out, omp_in)) &
  !$omp&  initializer( omp_priv = point( 0, 0 ) )

  type(point) :: minp = point( HUGE(0), HUGE(0) ), maxp = point( 0, 0 )
  integer :: i

  !$omp parallel do reduction(min:minp) reduction(max:maxp)
  do i = 1, n
     call minproc(minp, points(i))
     call maxproc(maxp, points(i))
  end do
  print *, "min = (", minp%x, minp%y, ")"
  print *, "max = (", maxp%x, maxp%y, ")"

 contains
  subroutine minproc ( out, in )
    implicit none
    type(point), intent(inout) :: out
    type(point), intent(in) :: in

    out%x = min( out%x, in%x )
    out%y = min( out%y, in%y )
  end subroutine minproc

  subroutine maxproc ( out, in )
    implicit none
    type(point), intent(inout) :: out
    type(point), intent(in) :: in

    out%x = max( out%x, in%x )
    out%y = max( out%y, in%y )
  end subroutine maxproc

end subroutine



The following example shows the same computation as  _udr.1_  but it illustrates that you can craft complex expressions in the user-defined reduction declaration. In this case, instead of calling the  _minproc_  and  _maxproc_  functions we inline the code in a single expression.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.2.c
* type: C
* version: omp_4.0
*/
#include <stdio.h>
#include <limits.h>

struct point {
  int x;
  int y;
};

#pragma omp declare reduction(min : struct point : \
        omp_out.x = omp_in.x > omp_out.x  ? omp_out.x : omp_in.x, \
        omp_out.y = omp_in.y > omp_out.y  ? omp_out.y : omp_in.y ) \
        initializer( omp_priv = { INT_MAX, INT_MAX } )

#pragma omp declare reduction(max : struct point : \
        omp_out.x = omp_in.x < omp_out.x  ? omp_out.x : omp_in.x,  \
        omp_out.y = omp_in.y < omp_out.y  ? omp_out.y : omp_in.y ) \
        initializer( omp_priv = { 0, 0 } )

void find_enclosing_rectangle ( int n, struct point points[] )
{
  struct point minp = { INT_MAX, INT_MAX }, maxp = {0,0};
  int i;

#pragma omp parallel for reduction(min:minp) reduction(max:maxp)
  for ( i = 0; i < n; i++ ) {
    if ( points[i].x < minp.x ) minp.x = points[i].x;
    if ( points[i].y < minp.y ) minp.y = points[i].y;
    if ( points[i].x > maxp.x ) maxp.x = points[i].x;
    if ( points[i].y > maxp.y ) maxp.y = points[i].y;
  }
  printf("min = (%d, %d)\n", minp.x, minp.y);
  printf("max = (%d, %d)\n", maxp.x, maxp.y);
}



The corresponding code of the same example in Fortran is very similar except that the assignment expression in the `declare` `reduction` directive can only be used for a single variable, in this case through a type structure constructor  _point( ... )_ .

In [ ]:

! name: udr.2.f90
! type: F-free
! version: omp_4.0
module data_type

  type :: point
    integer :: x
    integer :: y
  end type

end module data_type

subroutine find_enclosing_rectangle ( n, points )
  use data_type
  implicit none
  integer :: n
  type(point) :: points(*)

  !$omp declare reduction( min : point :  &
  !$omp&   omp_out = point(min( omp_out%x, omp_in%x ), &
  !$omp&                   min( omp_out%y, omp_in%y )) ) &
  !$omp&   initializer( omp_priv = point( HUGE(0), HUGE(0) ) )

  !$omp declare reduction( max : point :  &
  !$omp&   omp_out = point(max( omp_out%x, omp_in%x ), &
  !$omp&                   max( omp_out%y, omp_in%y )) ) &
  !$omp&   initializer( omp_priv = point( 0, 0 ) )

  type(point) :: minp = point( HUGE(0), HUGE(0) ), maxp = point( 0, 0 )
  integer :: i

  !$omp parallel do reduction(min: minp) reduction(max: maxp)
  do i = 1, n
     minp%x = min(minp%x, points(i)%x)
     minp%y = min(minp%y, points(i)%y)
     maxp%x = max(maxp%x, points(i)%x)
     maxp%y = max(maxp%y, points(i)%y)
  end do
  print *, "min = (", minp%x, minp%y, ")"
  print *, "max = (", maxp%x, maxp%y, ")"

end subroutine



The following example shows the use of special variables in arguments for combiner (`omp_in` and `omp_out`) and initializer (`omp_priv` and `omp_orig`) routines.  This example returns the maximum value of an array and the corresponding index value. The `declare` `reduction` directive specifies a user-defined reduction operation  _maxloc_  for data type  _struct_   _mx_s_ . The function  _mx_combine_  is the combiner and the function  _mx_init_  is the initializer.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.3.c
* type: C
* version: omp_4.0
*/

#include <stdio.h>
#define N 100

struct mx_s {
   float value;
   int index;
};

/* prototype functions for combiner and initializer in
   the declare reduction */
void mx_combine(struct mx_s *out, struct mx_s *in);
void mx_init(struct mx_s *priv, struct mx_s *orig);

#pragma omp declare reduction(maxloc: struct mx_s: \
        mx_combine(&omp_out, &omp_in)) \
        initializer(mx_init(&omp_priv, &omp_orig))

void mx_combine(struct mx_s *out, struct mx_s *in)
{
   if ( out->value < in->value ) {
      out->value = in->value;
      out->index = in->index;
   }
}

void mx_init(struct mx_s *priv, struct mx_s *orig)
{
   priv->value = orig->value;
   priv->index = orig->index;
}

int main(void)
{
   struct mx_s mx;
   float val[N], d;
   int i, count = N;

   for (i = 0; i < count; i++) {
      d = (N*0.8f - i);
      val[i] = N * N - d * d;
   }

   mx.value = val[0];
   mx.index = 0;
   #pragma omp parallel for reduction(maxloc: mx)
   for (i = 1; i < count; i++) {
      if (mx.value < val[i])
      {
         mx.value = val[i];
         mx.index = i;
      }
   }

   printf("max value = %g, index = %d\n", mx.value, mx.index);
   /* prints 10000, 80 */

   return 0;
}



Below is the corresponding Fortran version of the above example.  The `declare` `reduction` directive specifies the user-defined operation  _maxloc_  for user-derived type  _mx_s_ .  The combiner  _mx_combine_  and the initializer  _mx_init_  are specified as subprograms.

In [ ]:

! name: udr.3.f90
! type: F-free
! version: omp_4.0
program max_loc
   implicit none

   type :: mx_s
      real value
      integer index
   end type

   !$omp declare reduction(maxloc: mx_s: &
   !$omp&        mx_combine(omp_out, omp_in)) &
   !$omp&        initializer(mx_init(omp_priv, omp_orig))

   integer, parameter :: N = 100
   type(mx_s) :: mx
   real :: val(N), d
   integer :: i, count

   count = N
   do i = 1, count
      d = N*0.8 - i + 1
      val(i) = N * N - d * d
   enddo

   mx%value = val(1)
   mx%index = 1
   !$omp parallel do reduction(maxloc: mx)
   do i = 2, count
      if (mx%value < val(i)) then
         mx%value = val(i)
         mx%index = i
      endif
   enddo

   print *, 'max value = ', mx%value, ' index = ', mx%index
   ! prints 10000, 81

 contains

 subroutine mx_combine(out, in)
   implicit none
   type(mx_s), intent(inout) :: out
   type(mx_s), intent(in) :: in

   if ( out%value < in%value ) then
      out%value = in%value
      out%index = in%index
   endif
 end subroutine mx_combine

 subroutine mx_init(priv, orig)
   implicit none
   type(mx_s), intent(out) :: priv
   type(mx_s), intent(in) :: orig

   priv%value = orig%value
   priv%index = orig%index
 end subroutine mx_init

end program



The following example explains a few details of the user-defined reduction  in Fortran through modules. The `declare` `reduction` directive is declared in a module ( _data_red_ ).  The reduction-identifier  _.add._  is a user-defined operator that is to allow accessibility in the scope that performs the reduction operation. The user-defined operator  _.add._  and the subroutine  _dt_init_  specified in the `initializer` clause are defined in the same subprogram.

The reduction operation (that is, the `reduction` clause) is in the main program. The reduction identifier  _.add._  is accessible by use association. Since  _.add._  is a user-defined operator, the explicit interface should also be accessible by use association in the current program unit. Since the `declare` `reduction` associated to this `reduction` clause has the `initializer` clause, the subroutine specified on the clause must be accessible in the current scoping unit.  In this case, the subroutine  _dt_init_  is accessible by use association.

In [ ]:

! name: udr.4.f90
! type: F-free
! version: omp_4.0
module data_red
! Declare data type.
  type dt
    real :: r1
    real :: r2
  end type

! Declare the user-defined operator .add.
  interface operator(.add.)
    module procedure addc
  end interface

! Declare the user-defined reduction operator .add.
!$omp declare reduction(.add.:dt:omp_out=omp_out.add.omp_in) &
!$omp& initializer(dt_init(omp_priv))

 contains
! Declare the initialization routine.
  subroutine dt_init(u)
    type(dt) :: u
    u%r1 = 0.0
    u%r2 = 0.0
  end subroutine

! Declare the specific procedure for the .add. operator.
  function addc(x1, x2) result(xresult)
    type(dt), intent(in) :: x1, x2
    type(dt) :: xresult
    xresult%r1 = x1%r1 + x2%r2
    xresult%r2 = x1%r2 + x2%r1
  end function

end module data_red

program main
  use data_red, only : dt, dt_init, operator(.add.)

  type(dt) :: xdt1, xdt2
  integer :: i

  xdt1 = dt(1.0,2.0)
  xdt2 = dt(2.0,3.0)

! The reduction operation
!$omp parallel do reduction(.add.: xdt1)
  do i = 1, 10
    xdt1 = xdt1 .add. xdt2
  end do
!$omp end parallel do

  print *, xdt1

end program



The following example uses user-defined reductions to declare a plus (+) reduction for a C++ class. As the `declare` `reduction` directive is inside the context of the  _V_  class the expressions in the `declare` `reduction` directive are resolved in the context of the class. Also, note that the `initializer` clause uses a copy constructor to initialize the private variables of the reduction and it uses as parameter to its original variable by using the special variable `omp_orig`.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.5.cpp
* type: C++
* version: omp_4.0
*/
class V {
   float *p;
   int n;

public:
   V( int _n )     : n(_n)  { p = new float[n]; }
   V( const V& m ) : n(m.n) { p = new float[n]; }
   ~V() { delete[] p; }

   V& operator+= ( const V& );

   #pragma omp declare reduction( + : V : omp_out += omp_in ) \
           initializer(omp_priv(omp_orig))
};



The following examples shows how user-defined reductions can be defined for some STL containers. The first `declare` `reduction` defines the plus (+) operation for  _std::vector<int>_  by making use of the  _std::transform_  algorithm. The second and third define the merge (or concatenation) operation for  _std::vector<int>_  and  _std::list<int>_ .  It shows how the user-defined reduction operation can be applied to specific data types of an STL.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.6.cpp
* type: C++
* version: omp_4.0
*/
#include <algorithm>
#include <list>
#include <vector>

#pragma omp declare reduction( + : std::vector<int> : \
     std::transform (omp_out.begin(), omp_out.end(),  \
        omp_in.begin(), omp_in.end(),std::plus<int>()))

#pragma omp declare reduction( merge : std::vector<int> : \
     omp_out.insert(omp_out.end(), omp_in.begin(), omp_in.end()))

#pragma omp declare reduction( merge : std::list<int> : \
     omp_out.merge(omp_in))

